In [ ]:
%matplotlib inline
import numpy as np
import pylab as pl
import sympy as sym
from sympy.functions import Abs
#from sympy import Abs, Symbol, S

The estimate we want to check:

$$
|| f -p || \leq ||f^{n+1}||_\infty \frac{||w(x)||_\infty}{(n+1)!}
$$

in order to do so we need to define, simbolic and numerical functions. [`Sympy`]() is a very useful package to handle symbolic expressions and to export them to numerical functions. A the beginnnig of this notebook it is imported with the command: `import sympy as sym`. Let's define a simple function:

**Hint:** Try to plat around, derivatives integrals...

In [ ]:
t = sym.symbols('t', real=True)
sf = sym.sin(2.*sym.pi*t) # sf, simbolic function.
print(sf)

To make this function *digestible* by numpy we use the simple command `nf = sym.lambdify(t,f, 'numpy')`

In [ ]:
f = sym.lambdify(t,sf, 'numpy')
print(f)
print(f(np.array([0,.25])))

We aim at controlling all of the pieces of the inequality above, plot how terms behave with the degree, and see what happens `:)`

Good thing is to start from the beginning and control the term $||f-p||_\infty$. We recall that:
$$
p = \mathcal{L}^n f := \sum_{i=0}^n f(x_i) l^n_i(x),
$$

with

$$
l^n_i(x) := \prod_{j=0, j\neq i}^n \frac{(x-x_j)}{(x_i-x_j)}  \qquad
i = 0, \dots, n.
$$

Let's implment this guy. We want to fill the matrix `Ln` with $n+1$ rows and as many colums as the number of points where we evaluate the funtion. A good idea would be to collect the few operations in a function, like this one:
```python
def lagrangian_interpolation_matrix(x,q):
    ...
    return Ln
```
so that we can recalli whenever we need it.

**Hint:** I wouldn't call myself a good programmer, but I do my best to be like that. First construct the code in the main section of your program, run it, check that it works, then collect the precious commmands you wrote in an function.

### Step 0

```python
n = 3
q = np.linspace(0,1,n+1)

x = np.linspace(0,1,2**4+1)

Ln = np.zeros((n+1, len(x)))

for i in xrange(n+1):
    Ln[i] = np.product([ (x-q[j])/(q[i]-q[j]) for j in xrange(n+1) if j != i], axis=0)

_ = pl.plot(x,Ln.T)
```

In [ ]:
def lagrangian_interpolation_matrix(x,q):
    n = q.shape[0]-1
    Ln = np.zeros((n+1, len(x)))

    for i in xrange(n+1):
        Ln[i] = np.product([ (x-q[j])/(q[i]-q[j]) for j in xrange(n+1) if j != i], axis=0)
        
    return Ln

In [ ]:
n = 3
q = np.linspace(0,1,n+1)

x = np.linspace(0,1,2**5+1)

Ln = lagrangian_interpolation_matrix(x,q)

_ = pl.plot(x,Ln.T)

From the previous lecture we know that the mathemathical expression:
$$
\mathcal{L}^nf := \sum_{i=0}^n f(x_i) l^n_i(x) 
$$

Can be easyly translated into the `numpy` line:
```python
Ln.T.dot(f(x))
```
Let's give it a try:


In [ ]:
#print(Ln.shape)
#print(f(x).shape)
_ = pl.plot(x,Ln.T.dot(f(q)))
_ = pl.plot(x,f(x),'g')

Ok this works, but wouldn't it be good to have a code that looks like mathemathics? In this way it should be easer to debug.. Mathemattics looks like: $\mathcal{L}^nf$. And code:

In [ ]:
Lf = lambda x : lagrangian_interpolation_matrix(x,q).T.dot(f(q))

_ = pl.plot(x,Lf(x)) # isn't this looking better?
_ = pl.plot(x,f(x),'g')

We need a very simple function to evaluate the norm infinity of a function $||\cdot||_\infty$:

In [ ]:
def linfty_norm(fun,a=0,b=1,nn=12):
    xx = np.linspace(a,b,2**nn+1)
    return np.amax(np.absolute(fun(xx)))

$||f-p||_\infty = $ `error`

$||f^{n+1}||_\infty = $ `nth_der`

$w(x) =\prod_{i=0}^n (x-q_i), \quad ||w(x)||_\infty = $ `w`

In [ ]:
w = lambda x : np.product([x-q[i] for i in xrange(n+1)], axis = 0)

If I didn't mess the code this a good spot to play aroud with the function to be checked:

In [ ]:
#sf = Abs(t-.5)#1/(1+(10*t-5)**2)# sf = sym.sin(2.*sym.pi*t)
#f = sym.lambdify(t,sf, 'numpy')
#_ = pl.plot(x,f(x),'b')
#_ = pl.plot(x,Lf(x),'g')

In [ ]:
error = lambda x : f(x) - Lf(x)

test_degrees = np.arange(2,10)
test_errors = []
test_nth_der = []
test_w = []
test_n_factorial = []

for n in test_degrees:
    q = np.linspace(0,1,n+1)
    test_errors.append(linfty_norm(error))
    der_sf = sym.diff(sf,'t',n+1)
    der_f = sym.lambdify(t,der_sf, 'numpy')
    test_nth_der.append(linfty_norm(der_f))
    test_n_factorial.append(np.math.factorial(n))
    test_w.append(linfty_norm(w))
    #np.math.factorial(n)

estimate = np.array(test_nth_der)/np.array(np.array(test_n_factorial))*np.array(test_w)


print(test_nth_der)
    

h1 = pl.semilogy(test_degrees,test_errors,label = 'Error')
h2 = pl.semilogy(test_degrees,test_nth_der,label = 'Derivative')
h3 = pl.semilogy(test_degrees,test_n_factorial, label = 'Factorial')
h3 = pl.semilogy(test_degrees,test_w, label = 'w(x)')
h4 = pl.semilogy(test_degrees,estimate, label = 'estimate')
_ = pl.legend()
